## Template optimisation salome_meca

Ce template s'appuie sur un exemple pour construire toutes les fonctions (optimisation d'une table modélisée en shell). Les fonctions notamment de parsing sont donc à adapter en fonction du cas d'utilisation.

In [6]:
# Modules utilisés

import subprocess
import numpy as np
import matplotlib.pyplot as plt


### Actualisation des variables

On prend en entrée ici un fichier texte pour venir stocker l'évolution des variables durant l'optimisation.

In [1]:
# Définir en premier lieu les noms des différents paramètres, pour que salome_meca puisse facilement les lire.

noms_params = ['L1', 'L2', 'H', 'P', 'a', 'N'] # Exemple de noms

# Fonction d'écriture, faite pour être lisible par salome_meca

def write_params(self, params):
    list_params = [(self.noms_params[i], params[i]) for i in range(len(params))]
    filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/ModeleTable_OCTAVE/Param_test.txt' # Emplacement du fichier d'écriture des paramètres actuels
    with open(filename, 'w') as file:
        for param, value in list_params:
            file.write(f"{param} {value}\n")
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_para_compo.txt' # Emplacement du fichier de sauvegarde des paramètres au fur et à mesure de l'optimisation
    with open(file_path, 'a') as file:
        file.write('-------\n')
        for param, value in list_params:
            file.write(f"{param} {value}\n")

### Simulations avec salome_meca et code aster

In [4]:
# Calcul via code aster. Bien vérifier la commande donnée dans subprocess.check_output; dans le fichier contenant le export, il faut avoir les fichiers de résultats du calcul, 
# le fichier.comm et les autres fichiers comme les erreurs (message, launch)

def launch_calcul():
    l1 = subprocess.check_output('wsl -d smeca /opt/sm/bin/as_run /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/export')
    return l1

# Maillage via salome_meca. Celui-ci se fait via lancement à distance d'un script qui est soit écrit à la main soit obtenu en faisant un "dump study" dans salome_meca.
# Bien vérifier la commande également surtout le chemin d'accès au script.

def launch_mesh():
    l1 = subprocess.check_output('wsl -d smeca /opt/sm/bin/salome_meca shell python /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/script_compo2.py')
    return l1

### Parsing intelligent des résultats des calculs code aster

Ces fonctions dépendent de vos sorties obtenues par code aster. Ici, on donne des exemples de fonctions écrites.

In [9]:
# Lecture d'un fichier contenant les déformations; ici on s'intéresse aux déformations selon X.

def read_defo():
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/deformations' # Fichier localisé des déformations
    defo_values = []
    start_reading = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if "EPXX" in line:
                start_reading = True
            if start_reading:
                if '=====>' in line:
                    break
                try:
                    defo = line.split("|")
                    if len(defo) <= 5:
                        break
                    else:
                        defo = float(defo[4])
                        defo_values.append(defo)
                except ValueError:
                    continue
    return defo_values

# Lecture d'un fichier contenant l'évaluation aux points du maillage de la contrainte de Von Mises.

def read_output():
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/res_const.txt' # Fichier localisé des contraintes
    vm_values = []
    start_reading = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if "VMIS" in line:
                start_reading = True

            if start_reading:
                if '=====>' in line:
                    break
                try:
                    vm = line.split("|")
                    if len(vm) <= 5:
                        break
                    else:
                        vm = float(vm[4])
                        vm_values.append(vm)
                except ValueError:
                    continue
    return vm_values

# Lecture des modes de flambement d'une structure.

def read_flamb():
    with open('C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/resu_flamb', 'r') as file: # Fichier localisé des modes
        lines = file.readlines()

    char_crit_values = []

    for line in lines:
        line = line.strip()
        if "=====>" in line:
            break
        if line and not any(c.isalpha() for c in line.split()[1:]):
            try:
                char_crit = float(line.split()[1].replace('D', 'E'))
                char_crit_values.append(char_crit)
            except (IndexError, ValueError):
                continue

    return np.array(char_crit_values)

# Lecture de la masse de la structure étudiée

def read_mass():
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/masse.txt' # Fichier localisé de la masse
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Table"):
            columns = line.split()
            try:
                return float(columns[2])
            except (IndexError, ValueError) as e:
                raise ValueError("Erreur lors de l'extraction de la masse : {}".format(e))

    raise ValueError("La ligne contenant la masse n'a pas été trouvée.")



### Ecriture dans le fichier .comm de code aster : modifier les commandes pour par exemple actualiser d'autres paramètres ou variables

In [10]:
# Fonction destinée à écrire les variables 'a' et 'N' dans le fichier code aster au fur et à mesure de l'optimisation. Les variables peuvent être insérées n'importe où,
# le plus simple est au début du fichier .comm.

def insert_line_before_clt140(self, params):
    filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/Stage_1.comm'
    with open(filename, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if 'a = ' in line:
            lines[i] = f'a = {params[len(params)-2]}\n'
            break

    for i, line in enumerate(lines):
        if 'N = ' in line:
            lines[i] = f'N = {params[len(params)-1]}\n'
            break

    with open(filename, 'w') as file:
        file.writelines(lines)

# Fonction destinée à retirer une partie du code dans le fichier .comm (la fonction suivante réécrit ces lignes ensuite).

def remove_section():
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/Stage_1.comm'
    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_keyword = 'clt140 = ' # Mot de début de la section à supprimer, à déterminer manuellement dans votre fichier .comm
    end_keyword = 'fieldmat = ' # Mot de fin de la section à supprimer
    new_lines = []
    inside_section = False

    for line in lines:
        if start_keyword in line:
            new_lines.append(line)
            inside_section = True
        elif end_keyword in line and inside_section:
            inside_section = False
            new_lines.append(line)
        elif not inside_section:
            new_lines.append(line)

    with open(file_path, 'w') as file:
        file.writelines(new_lines)

# Fonction pour réécrire les lignes effacées. Ici, on a effacé les lignes qui contenaient les couches dans un matériau composite,
# pour les réécrire en fonction de l'actualisation du paramètre N

def for_couches( params):
    filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/Stage_1.comm'
    with open(filename, 'r') as file:
        lines = file.readlines()
    b = True
    # Texte que l'on veut écrire; bien faire attention car au moindre espace manquant dans la syntaxe, code aster ne comprend plus la ligne de commande.
    
    to_bewritten = f'''_F(EPAIS=5.0, 
                                MATER=wood, 
                                ORIENTATION={params[-2]}),'''
    for i in range(int(params[len(params) - 1] - 1)):
        if b:
            to_bewritten += f'''\n                                 _F(EPAIS=5.0, 
                                MATER=wood, 
                                ORIENTATION={params[-2]}),'''
            b = False
        else:
            to_bewritten += f'''\n                                 _F(EPAIS=5.0, 
                                MATER=wood, 
                                ORIENTATION=0.0),'''
            b = True
    for k, line in enumerate(lines):
        if 'clt140 = ' in line:
            index = k
            break
    lines[index] = f'''clt140 = DEFI_COMPOSITE(identifier='7:1', 
                    COUCHE=({to_bewritten}))\n'''

    with open(filename, 'w') as file:
        file.writelines(lines)

### Fonction objectif et contraintes.

In [11]:
# Fonction objectif à optimiser, ici on regarde deux objectifs : masse et déformations

def objective( params):
    insert_line_before_clt140(params)
    write_params(params)
    remove_section()
    for_couches(params)
    launch_mesh()
    launch_calcul()
    mass = read_mass()
    defo = read_defo()

    alpha = min(np.abs(defo))

    # Ecriture des valeurs actuelles des objectifs pour enregistrer leur évolution
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_defo_compo.txt'
    with open(file_path, 'a') as file:
        file.write('-------\n')
        file.write(f"{alpha}\n")
    file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_obj_compo.txt'
    with open(file_path, 'a') as file:
        file.write('-------\n')
        file.write(f"{mass}\n")
    return mass , alpha 

In [12]:
# Contraintes, nécessite la définition ici du Re pour Von Mises

Re = 80 # Ici Re du bois, tout en MPa

def constr_flamb():
    results_flamb = read_flamb()
    coeff = np.abs(results_flamb)
    return min(coeff) - 1

def constr_vm():
    results_vm  = read_output()
    coeff = np.abs(results_vm)
    return -max(coeff) + Re


### Classe globale pour faciliter l'optimisation

In [13]:
# Classe qui regroupe toutes les fonctions précédentes pour faciliter la lisibilité.

class MyModel2:
    def __init__(self, noms_params, Re, penalty, epaiss, rho):
        self.noms_params = noms_params
        self.Re = Re
        self.penalty = penalty
        self.epaiss = epaiss
        self.rho = rho

    def write_params(self, params):
        list_params = [(self.noms_params[i], params[i]) for i in range(len(params))]
        filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/ModeleTable_OCTAVE/Param_test.txt'
        with open(filename, 'w') as file:
            for param, value in list_params:
                file.write(f"{param} {value}\n")
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_para_compo.txt'
        with open(file_path, 'a') as file:
            file.write('-------\n')
            for param, value in list_params:
                
                file.write(f"{param} {value}\n")
    def read_defo(self):
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/deformations'
        defo_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "EPXX" in line:
                    start_reading = True
            
                if start_reading:

                    if '=====>' in line:
                        break
                    try:
                        defo = line.split("|")
                        if len(defo) <=5:
                            break
                        else:
                            defo = defo[4]
                            defo = float(defo)
                            defo_values.append(defo)
                    except ValueError:
                        continue
        return(defo_values)
        
    def read_output(self):
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/res_const.txt'
        vm_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "VMIS" in line:
                    start_reading = True
            
                if start_reading:

                    if '=====>' in line:
                        break
                    try:
                        vm = line.split("|")
                        if len(vm) <=5:
                            break
                        else:
                            vm = vm[4]
                            vm = float(vm)
                            vm_values.append(vm)
                    except ValueError:
                        continue
        return(vm_values)

    def launch_calcul(self):
        l1 = subprocess.check_output('wsl -d smeca /opt/sm/bin/as_run /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/export')
        return(l1)        

    def launch_mesh(self):
        l1 = subprocess.check_output('wsl -d smeca /opt/sm/bin/salome_meca shell python /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/script_compo2.py')
        return(l1)

    def read_flamb(self):
        with open('C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/resu_flamb', 'r') as file:
            lines = file.readlines()

        # Initialiser une liste pour stocker les valeurs de CHAR_CRIT
        char_crit_values = []

        # Parcourir chaque ligne et extraire la valeur de CHAR_CRIT
        for line in lines:
            # Supprimer les espaces en début et fin de ligne
            line = line.strip()
            if "=====>" in line:
                break
            # Vérifier si la ligne contient une valeur CHAR_CRIT valide
            if line and not any(c.isalpha() for c in line.split()[1:]):
                # Extraire la valeur de CHAR_CRIT (colonne 2)
                try:
                    char_crit = line.split()[1].replace('D', 'E')  # Remplacer D par E pour notation scientifique
                    char_crit = float(char_crit)  # Convertir en float
                    char_crit_values.append(char_crit)
                except (IndexError, ValueError):
                # Ignorer les lignes qui ne correspondent pas au format attendu
                    continue

        char_crit_array = np.array(char_crit_values)
        return(char_crit_array)

    def read_mass(self):
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/masse.txt'
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        for line in lines:
            if line.startswith("Table"):
                columns = line.split()
                try:
                    mass = float(columns[2])
                       
                    return mass
                except (IndexError, ValueError) as e:
                    raise ValueError("Erreur lors de l'extraction de la masse : {}".format(e))
         
        raise ValueError("La ligne contenant la masse n'a pas été trouvée.")
    def insert_line_before_clt140(self, params):
        filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/Stage_1.comm'
        # Lire le contenu du fichier
        with open(filename, 'r') as file:
            lines = file.readlines()
    
        # Trouver l'index de la ligne contenant 'clt140'
        for i, line in enumerate(lines):
            if 'a = ' in line:
                index = i
                break    
        lines[index] =  f'a = {params[len(params)-2]}\n'
        for i, line in enumerate(lines):
            if 'N = ' in line:
                index = i
                break    
        lines[index] =  f'N = {params[len(params)-1]}\n'    
        # Écrire le contenu modifié dans le fichier
        with open(filename, 'w') as file:
            file.writelines(lines)
    
    def remove_section(self):
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/Stage_1.comm'
        with open(file_path, 'r') as file:
            lines = file.readlines()
        start_keyword = 'clt140 = '
        end_keyword = 'fieldmat = '
        new_lines = []
        inside_section = False
        for line in lines:
            if start_keyword in line:
                new_lines.append(line)
                inside_section = True
            if end_keyword in line and inside_section == True:
                inside_section = False
                new_lines.append(line)
                continue
            if inside_section == False:
                new_lines.append(line)
        with open(file_path, 'w') as file:
            file.writelines(new_lines)
    
    def for_couches(self, params):
        filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table_composite_axe_Files/RunCase_5/Result-Stage_1/Stage_1.comm'

        with open(filename, 'r' ) as file:
            lines = file.readlines()
        b = True
        to_bewritten = f'''_F(EPAIS=5.0, 
                                    MATER=wood, 
                                    ORIENTATION={params[-2]}),'''
        for i in range (int(params[len(params)-1]-1)):
            if b :
                to_bewritten = to_bewritten + '\n'
                to_bewritten = to_bewritten + f'''                                 _F(EPAIS=5.0, 
                                    MATER=wood, 
                                    ORIENTATION={params[-2]}),'''
                b = False
            else : 
                to_bewritten = to_bewritten + '\n'
                to_bewritten = to_bewritten + '''                                 _F(EPAIS=5.0, 
                                    MATER=wood, 
                                    ORIENTATION=0.0),'''
                b = True
        for k, line in enumerate(lines):
            if 'clt140 = ' in line:
                index = k
                break 
        lines[index] = f'''clt140 = DEFI_COMPOSITE(identifier=\'7:1\', 
                        COUCHE= ({to_bewritten}))\n'''
        with open(filename, 'w') as file:
                file.writelines(lines)
        
    def perform_calculations(self, params):
        self.insert_line_before_clt140(params)
        self.write_params(params)
        self.remove_section()
        self.for_couches(params)
        self.launch_mesh()
        self.launch_calcul()
 
        self.results_vm = self.read_output()
        self.results_flamb = self.read_flamb()
        self.mass = self.read_mass()
        self.defo = self.read_defo()   

    def objective(self, params):
        self.perform_calculations(params)
        alpha = min(np.abs(self.defo))
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_defo_compo.txt'
        with open(file_path, 'a') as file:
            file.write('-------\n')
            file.write(f"{alpha}\n")
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_obj_compo.txt'
        with open(file_path, 'a') as file:
            file.write('-------\n')
            file.write(f"{self.mass}\n")
        return self.mass,alpha
    
    def constr_flamb(self, params):
        self.perform_calculations(params)
        coeff = np.abs(self.results_flamb)

        return(min(coeff)-1)
    
    def constr_vm(self, params):
        self.perform_calculations(params)
        coeff = np.abs(self.results_vm)
        return(-max(coeff)+Re)

### Fonctions annexes de traçage et de mise à zéro des fichiers d'historique

In [16]:
def clear_file(file_path):
    with open(file_path, 'w') as file:
        # Ouvrir en mode écriture 'w' efface le contenu du fichier
        pass 

def trace(file_path):
    values = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line and line!='-------' :
                if len(values) == 0:
                    values.append(float(line))
                elif float(line)!=values[len(values)-1]: 
                    values.append(float(line))
    return values


### Boucle d'optimisation avec OpenTurns

In [14]:
# Ici, on est en mutli-objectif, on utilise l'algorithme NSGAII de Pagmo pour résoudre. On pourrait faire de même avec d'autres algorithmes.
import openturns as ot
# Créer une instance du modèle, paramètres à adapter en fonction de la situation et votre étude
noms_params = ['L1', 'L2', 'H', 'P', 'a', 'N']
Re = 80
penalty = 10e6 # si on veut optimiser par pénalisation, pas le cas ici
epaiss = 20 # Inutile ici
rho = 7800 

model = MyModel2(noms_params, Re, penalty, epaiss, rho)

# Définir la fonction objectif pour OpenTURNS
def objective_function(params):
    return [model.objective(params)[0], model.objective(params)[1]]

def constraint_flamb(params):
    return([model.constr_flamb(params)])

def constraint_vm(params):
    return([model.constr_vm(params)])

# Création de la pop initiale pour l'algo génétique (pour la variable entière)
integer_values = [[i] for i in range (1,20)]
discrete_values = ot.Sample(integer_values)
factory = ot.UserDefinedFactory()
distribution = factory.build(discrete_values)

# Créer un problème d'optimisation
problem = ot.OptimizationProblem(ot.PythonFunction(len(noms_params), 2, objective_function))
problem.setMinimization(True)

# bornes des variables
bounds = ot.Interval([500,300,400,25,0,1],[3000, 2000, 1500, 100, 90,20]) 
problem.setBounds(bounds)

# Contraintes
problem.setInequalityConstraint(ot.PythonFunction(len(noms_params), 1, constraint_flamb) )
problem.setInequalityConstraint(ot.PythonFunction(len(noms_params), 1, constraint_vm) )

# si les variables sont entières ou pas
problem.setVariablesType([ot.OptimizationProblemImplementation.CONTINUOUS,ot.OptimizationProblemImplementation.CONTINUOUS,ot.OptimizationProblemImplementation.CONTINUOUS,
                           ot.OptimizationProblemImplementation.CONTINUOUS,ot.OptimizationProblemImplementation.CONTINUOUS,ot.OptimizationProblemImplementation.INTEGER]) 

In [15]:
uniform = ot.ComposedDistribution([ot.Uniform(500, 3000), ot.Uniform(300, 2000), ot.Uniform(400, 1500), ot.Uniform(25,100), ot.Uniform(0,90), distribution])

ot.RandomGenerator.SetSeed(0)

init_pop = uniform.getSample(8) # nombre d'individus dans la population, influe sur le nombre d'appels au modèle.

In [ ]:
# Calcul en tant que tel

file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_para_compo.txt'
clear_file(file_path)
file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_obj_compo.txt'
clear_file(file_path)
file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_defo_compo.txt'
clear_file(file_path)

algo = ot.Pagmo(problem, 'nsga2', init_pop) 

algo.setMaximumIterationNumber(5) # à ajuster en fonction du coût computationnel  

algo.run() 

result = algo.getResult() 

final_pop_x = result.getFinalPoints() 

final_pop_y = result.getFinalValues() 

In [ ]:
# Extraction du front de Pareto

front0 = result.getParetoFrontsIndices()[0] 

front0_x = final_pop_x.select(front0) 

front0_y = final_pop_y.select(front0) 

In [ ]:
# Tracé du front de Pareto
plt.plot(front0_y[:,0])
plt.plot(front0_y[:,1])

In [ ]:
# Tracé de la masse

Mass_tab = trace('C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/compo/evol_obj_compo.txt')

plt.plot(Mass_tab)
plt.xlabel('Nombre d\'appels')
plt.ylabel('Valeur')